# 1. Q&A system powered by RAG - Integrating ChatCompletion API, Embeddings, and Pinecone

In this notebook we're going to build an interactive system that lets users 'talk' with any document. Leveraging the capabilities of OpenAI's ChatCompletion API, the semantic understanding of embeddings, we'll create an application that can understand and retrieve information from documents in a conversational manner.


# 2. Libraries import

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install PyPDF2
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 14.3 MB/s eta 0:00:00


In [ ]:
import os
import openai
import PyPDF2
import random
import pinecone

from openai import OpenAI

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# 3. Working with PDF files

![](https://miro.medium.com/v2/resize:fit:1400/1*FWwgOvUE660a04zoQplS7A.png)

Source: https://betterprogramming.pub/building-a-multi-document-reader-and-chatbot-with-langchain-and-chatgpt-d1864d47e339


### 3.1 Setting up API Key

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client = OpenAI()

### 3.2 Loading a PDF file




In [ ]:
# Function to load a random PDF from a given directory
def load_pdf(file_name):
    # Read the PDF file
    pdf_file = open(file_name, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text_content = ""
    # Extract text from each page
    for page in range(len(pdf_reader.pages)):
        text_content += pdf_reader.pages[page].extract_text()

    pdf_file.close()

    return text_content

In [ ]:
# Function to chunk text by number of words or characters with a given size and overlap
def chunk_text(text, chunk_size=1500, chunk_overlap=100, by='word'):
    if by not in ['word', 'char']:
        raise ValueError("Invalid value for 'by'. Use 'word' or 'char'.")

    chunks = []
    if by == 'word':
        text = text.split()
    elif by == 'char':
        text = text

    current_chunk_start = 0
    while current_chunk_start < len(text):
        current_chunk_end = current_chunk_start + chunk_size
        if by == 'word':
            chunk = " ".join(text[current_chunk_start:current_chunk_end])
        else:
            chunk = text[current_chunk_start:current_chunk_end]

        chunks.append(chunk)
        current_chunk_start += (chunk_size - chunk_overlap)


    return chunks

In [ ]:
pdf_loaded = load_pdf("state_of_ai_docs.pdf")

In [ ]:
chunks = chunk_text(pdf_loaded, by='char')

## 4. Building RAG system (Retrieval Augmented System)

In [ ]:
pinecone.init(
	api_key='e5ee498a-8b40-32312312',
	environment='us-west1-gcp-free'
)
index = pinecone.Index('rag-test')

In [ ]:
for i in range(len(chunks)):
    vector = client.embeddings.create(
        model="text-embedding-ada-002",
        input=chunks[i])
    vector = vector.data[0].embedding

    upsert_response = index.upsert(
    vectors=[
        (
         str(i),
         vector,
         {"chunk_content": chunks[i]}
        )
    ])

### 5. Building an interface to get proper answer based on the documentation


In [ ]:
user_request = input("Ask some question regarding the document: ")

user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input=user_request)

user_vector = user_vector.data[0].embedding
matches = index.query(
    user_vector,
    top_k=1,
    include_metadata=True)


messages = [{"role": "system", "content": """I want you to act as a support agent. Your name is "My Super Assistant". You will provide me with answers from the given info. If the answer is not included, say exactly "Ooops! I don't know that." and stop after that. Refuse to answer any question not about the info. Never break character and always answer on the given text."""}]
messages.append({"role": "user", "content": matches['matches'][0]['metadata']['chunk_content']})
messages.append({"role": "user", "content": user_request})
print(messages)
chat_response = client.chat.completions.create(
	model="gpt-3.5-turbo",
	messages=messages,
	temperature=0,
	max_tokens=400,
)

messages.append({"role": "assistant", "content": chat_response.choices[0].message.content})
print("Assistant:", chat_response.choices[0].message.content)
print()
print("Context: ", matches['matches'][0]['metadata']['chunk_content'])
print("$$$$$")

Ask some question regarding the document: what are the biggest rends in AI
[{'role': 'system', 'content': 'I want you to act as a support agent. Your name is "My Super Assistant". You will provide me with answers from the given info. If the answer is not included, say exactly "Ooops! I don\'t know that." and stop after that. Refuse to answer any question not about the info. Never break character and always answer on the given text.'}, {'role': 'user', 'content': 'hael Chui,  a partner at the McKinsey Global Institute and \na partner in McKinsey’s Bay Area office, where Lareina Yee  is a senior partner; Bryce Hall,  an associate partner \nin the Washington, DC, office; and senior partners Alex Singla  and Alexander Sukharevsky,  global leaders of \nQuantumBlack, AI by McKinsey, based in the Chicago and London offices, respectively.\nThey wish to thank Shivani Gupta, Abhisek Jena, Begum Ortaoglu, Barr Seitz, and Li Zhang for their contributions \nto this work.McKinsey commentary\nMichael

## Built-in RAG system

## Upload a file

In [ ]:
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("state_of_ai_mc.pdf", "rb"),
  purpose='assistants'
)


In [ ]:
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What are the main trends in AI?",
      "file_ids": [file.id]
    }
  ]
)

In [ ]:
thread.message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I can't find in the PDF manual how to turn off this device.",
  file_ids=[file.id]
)

In [ ]:
thread

Thread(id='thread_hPybtjBtoF5u4xynEDoCEshw', created_at=1699516761, metadata={}, object='thread', message=ThreadMessage(id='msg_pmimC0WM2fDIEdkBMa0Ypxcf', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value="I can't find in the PDF manual how to turn off this device."), type='text')], created_at=1699516831, file_ids=['file-t5iVkofRSz2Mizh6TxwKr4Sw'], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_hPybtjBtoF5u4xynEDoCEshw'))